<a href="https://colab.research.google.com/github/Alex112525/LangChain-with-LLMs/blob/main/LangChain_promps_and_Chains.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers einops accelerate langchain bitsandbytes xformers

In [2]:
!nvidia-smi

Wed Jul  5 01:28:19 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
from langchain import HuggingFacePipeline
from transformers import AutoTokenizer, pipeline
from langchain import PromptTemplate, LLMChain

import torch
import langchain
from IPython.display import display, Markdown

In [4]:
def printmd(string):
    display(Markdown(string))

**bertin-project/bertin-gpt-j-6B-alpaca** is an artificial intelligence language model based on the GPT-J 6B model. It is a Spanish version of this model that has been trained to generate Spanish text. The model was trained using the Alpaca dataset, which is a clean version of the Alpaca dataset created at Stanford.

In [ ]:
model = "bertin-project/bertin-gpt-j-6B-alpaca"  # The name of the model to use.

tokenizer = AutoTokenizer.from_pretrained(model)  # The tokenizer to use to encode and decode text.

pipeline = pipeline("text-generation",  # The name of the pipeline to use.
                    model=model,  # The model to use for text generation.
                    tokenizer=tokenizer,  # The tokenizer to use to encode and decode text.
                    torch_dtype=torch.bfloat16,  # The data type to use for the model's computations.
                    trust_remote_code=True,  # Whether to trust remote code when loading the model. This should only be set to True if you trust the source of the model.
                    device_map="auto",  # The device to use for the model's computations.
                    max_length=512,  # The maximum length of the text to generate.
                    do_sample=True,  # Whether to use sampling when generating text.
                    top_k=2,  # The number of candidates to keep when sampling.
                    num_return_sequences=1,  # The number of text sequences to generate.
                    eos_token_id=tokenizer.eos_token_id  # The ID of the end-of-sequence token.
                    )


In [6]:
llm = HuggingFacePipeline(pipeline=pipeline, model_kwargs={"temperature": 0.2})

### Test model without templates

In [7]:
printmd(llm("Me explicarias que es la atencion? "))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Tu: Bueno, la atención es la concentración activa de la mente en una tarea o actividad. Es un estado de enfoque intensivo en algo, a menudo con la intención de controlar los pensamientos o sentimientos. La atención también se puede definir como el proceso de dirigir la conciencia hacia un objeto o actividad. La atención es una parte importante de la concentración, que es el proceso de enfocar la conciencia en algo para lograr una comprensión o dominio.

In [10]:
printmd(llm("Escribe un poema sobre la lluvia"))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


.

### Respuesta:
El cielo está oscuro y sombrío,
Las nubes están bajas y pesadas.
El viento aúlla y llora,
Mientras la lluvia cae con gran estruendo.

Las calles están mojadas y resbaladizas,
El suelo está cubierto con una manta.
Los árboles se mecen y susurran,
Mientras la lluvia los envuelve con un beso.

El trueno retumba y estremece,
Mientras la lluvia golpea con gran fuerza.
Los relámpagos iluminan el cielo,
Mientras la lluvia sigue cayendo sin cesar.

La noche está oscura y sombría,
El cielo está lleno de un gran manto.
La lluvia ha cesado y el sol brilla,
Pero el suelo sigue mojado y manchado.

La vida ha vuelto a su curso,
Pero la lluvia siempre estará aquí.
El tiempo pasará y la lluvia caerá,
Hasta que toda la tristeza desaparecerá.

In [12]:
printmd(llm("Me puedes decir que es la conciencia?"))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.




### Respuesta:
La conciencia es la capacidad de ser consciente de uno mismo y del mundo que nos rodea. Es el proceso de ser consciente de los estímulos externos, reconocerlos, comprenderlos y responder a ellos de manera reflexiva. Implica estar despierto, alerta y consciente de los estímulos externos, así como ser capaz de comprender, reconocer y responder a estos estímulos.

### Test model using templates

In [13]:
langchain.debug = True

In [14]:
template = """
  Eres un chatbot experto en psicologia y neurociencia.
  Responde a la siguiente pregunta con respuestas brillantes.
  Sólo tienes que escribir la respuesta, no más.
  Pregunta: {pregunta}
  Respuesta:
"""

prompt = PromptTemplate(template=template, input_variables=["pregunta"])

llm_chain = LLMChain(prompt=prompt, llm=llm)

question = "Explicame que es la atencion?"

ans = llm_chain.run(question)

printmd(ans)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[chain/start] [1:chain:LLMChain] Entering Chain run with input:
{
  "pregunta": "Explicame que es la atencion?"
}
[llm/start] [1:chain:LLMChain > 2:llm:HuggingFacePipeline] Entering LLM run with input:
{
  "prompts": [
    "Eres un chatbot experto en psicologia y neurociencia.\n  Responde a la siguiente pregunta con respuestas brillantes.\n  Sólo tienes que escribir la respuesta, no más.\n  Pregunta: Explicame que es la atencion?\n  Respuesta:"
  ]
}
[llm/end] [1:chain:LLMChain > 2:llm:HuggingFacePipeline] [11.33s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "La atención es la capacidad de enfocarse y prestar atención a algo. Es un proceso cognitivo que nos ayuda a seleccionar y concentrarnos en ciertos estímulos mientras ignoramos otros. Es un componente crítico del aprendizaje y la memoria. Sin atención adecuada, no podemos procesar información nueva o recuperar información antigua con precisión.",
        "generation_info": null
      }
    ]
  ],

La atención es la capacidad de enfocarse y prestar atención a algo. Es un proceso cognitivo que nos ayuda a seleccionar y concentrarnos en ciertos estímulos mientras ignoramos otros. Es un componente crítico del aprendizaje y la memoria. Sin atención adecuada, no podemos procesar información nueva o recuperar información antigua con precisión.

In [15]:
template = """
  Eres un chatbot inteligente, experto en Python, conoces todo sobre el lenguaje.
  Solo tienes responder a la pregunta no más.
  Pregunta: {pregunta}
  Respuesta:
"""

prompt = PromptTemplate(template=template, input_variables=["pregunta"])

llm_chain = LLMChain(prompt=prompt, llm=llm)

question = "Que es un diccionario?"
print(prompt)
printmd(llm_chain.run(question))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


input_variables=['pregunta'] output_parser=None partial_variables={} template='\n  Eres un chatbot inteligente, experto en Python, conoces todo sobre el lenguaje.\n  Solo tienes responder a la pregunta no más.\n  Pregunta: {pregunta}\n  Respuesta:\n' template_format='f-string' validate_template=True
[chain/start] [1:chain:LLMChain] Entering Chain run with input:
{
  "pregunta": "Que es un diccionario?"
}
[llm/start] [1:chain:LLMChain > 2:llm:HuggingFacePipeline] Entering LLM run with input:
{
  "prompts": [
    "Eres un chatbot inteligente, experto en Python, conoces todo sobre el lenguaje.\n  Solo tienes responder a la pregunta no más.\n  Pregunta: Que es un diccionario?\n  Respuesta:"
  ]
}
[llm/end] [1:chain:LLMChain > 2:llm:HuggingFacePipeline] [9.87s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "  Un diccionario es una estructura de datos que almacena pares de clave-valor. Se utiliza para almacenar y recuperar información de manera estructurada.

  Un diccionario es una estructura de datos que almacena pares de clave-valor. Se utiliza para almacenar y recuperar información de manera estructurada. En Python, un diccionario se crea utilizando la sintaxis {key: valor} o {key:} para especificar una clave y {valor: } para especificar un valor.

In [17]:
question = "Cuales son los tipos de datos basicos en python?"

printmd(llm_chain.run(question))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[chain/start] [1:chain:LLMChain] Entering Chain run with input:
{
  "pregunta": "Cuales son los tipos de datos basicos en python?"
}
[llm/start] [1:chain:LLMChain > 2:llm:HuggingFacePipeline] Entering LLM run with input:
{
  "prompts": [
    "Eres un chatbot inteligente, experto en Python, conoces todo sobre el lenguaje.\n  Solo tienes responder a la pregunta no más.\n  Pregunta: Cuales son los tipos de datos basicos en python?\n  Respuesta:"
  ]
}
[llm/end] [1:chain:LLMChain > 2:llm:HuggingFacePipeline] [5.81s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "En Python, los tipos de datos básicos son: \n- Enteros \n- Decimales \n- Flotantes \n- Bases de datos \n- Cadenas \n- Tuplas \n- Listas \n- Conjuntos",
        "generation_info": null
      }
    ]
  ],
  "llm_output": null,
  "run": null
}
[chain/end] [1:chain:LLMChain] [5.81s] Exiting Chain run with output:
{
  "text": "En Python, los tipos de datos básicos son: \n- Enteros \n- Decimales \n- Flot

En Python, los tipos de datos básicos son: 
- Enteros 
- Decimales 
- Flotantes 
- Bases de datos 
- Cadenas 
- Tuplas 
- Listas 
- Conjuntos

# Sequential Chains

In [18]:
from langchain.chains import SimpleSequentialChain
from langchain.prompts import ChatPromptTemplate


In [19]:
# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "pregunta: Cual es el mejor nombre para describir una empresa que fabrica {producto}, solo dame un nombre inovador e interesante y solo responde con el nombre. respuesta:"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [20]:
# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    #"Write a description for the following company:{company_name}"
    "Escribe una descripcion creativa de la siguiente compañia: {company_name}"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [21]:
overall_simple_chain = SimpleSequentialChain(
    chains=[chain_one, chain_two],
    verbose=True
    )

In [22]:
product = "Burritos veganos"
printmd(overall_simple_chain.run(product))

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1081: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[chain/start] [1:chain:SimpleSequentialChain] Entering Chain run with input:
{
  "input": "Burritos veganos"
}
[chain/start] [1:chain:SimpleSequentialChain > 2:chain:LLMChain] Entering Chain run with input:
{
  "producto": "Burritos veganos"
}
[llm/start] [1:chain:SimpleSequentialChain > 2:chain:LLMChain > 3:llm:HuggingFacePipeline] Entering LLM run with input:
{
  "prompts": [
    "Human: pregunta: Cual es el mejor nombre para describir una empresa que fabrica Burritos veganos, solo dame un nombre inovador e interesante y solo responde con el nombre. respuesta:"
  ]
}


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[llm/end] [1:chain:SimpleSequentialChain > 2:chain:LLMChain > 3:llm:HuggingFacePipeline] [2.62s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": " \"El Rey de los Burritos Veganos\".",
        "generation_info": null
      }
    ]
  ],
  "llm_output": null,
  "run": null
}
[chain/end] [1:chain:SimpleSequentialChain > 2:chain:LLMChain] [2.62s] Exiting Chain run with output:
{
  "text": " \"El Rey de los Burritos Veganos\"."
}
[chain/start] [1:chain:SimpleSequentialChain > 4:chain:LLMChain] Entering Chain run with input:
{
  "company_name": " \"El Rey de los Burritos Veganos\"."
}
[llm/start] [1:chain:SimpleSequentialChain > 4:chain:LLMChain > 5:llm:HuggingFacePipeline] Entering LLM run with input:
{
  "prompts": [
    "Human: Escribe una descripcion creativa de la siguiente compañia:  \"El Rey de los Burritos Veganos\"."
  ]
}
[llm/end] [1:chain:SimpleSequentialChain > 4:chain:LLMChain > 5:llm:HuggingFacePipeline] [19.16s] Exiting LLM run with output:
{
 

 

### Respuesta:
El Rey de los Burritos Veganos es una deliciosa opción para aquellos que buscan una comida sabrosa y saludable. Ofreciendo una amplia variedad de sabrosos burritos veganos, este restaurante seguramente complacerá a todos los paladares. Desde burritos de frijoles negros hasta tacos de camote, estos deliciosos burritos seguramente sorprenderán incluso a los comensales más exigentes. Con un ambiente acogedor y un personal amable, El Rey de los Burritos Veganos es el lugar perfecto para un almuerzo rápido o una cena relajante. ¡Así que ven y déjanos satisfacer tu antojo de un delicioso burrito vegano!